In [20]:
import time, psutil, numpy as np
from pymilvus import connections, Collection

# 1. Configurações de Conexão (Local)
MILVUS_HOST = "localhost"
MILVUS_PORT = "19540" # Porta mapeada no seu Docker local
COLLECTION_NAME = "lfw_cosface_modular"

print(f"-> Conectando ao Milvus em {MILVUS_HOST}:{MILVUS_PORT}...")
connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)

# 2. Definir a coleção para este kernel
collection = Collection(COLLECTION_NAME)
print(f"✓ Coleção '{COLLECTION_NAME}' vinculada.")

# 3. CARREGAR OS DADOS (Necessário para a busca)
# Usando a pasta 'recursos' conforme sua alteração
print("-> Carregando embeddings de teste para a memória...")
try:
    emb_data = np.load("recursos/lfw_resnet_embeddings.npy").astype('float32')
    print(f"✓ {len(emb_data)} embeddings carregados com sucesso de 'recursos/'.")
except FileNotFoundError:
    print("❌ ERRO: Arquivo não encontrado em 'recursos/lfw_resnet_embeddings.npy'.")

-> Conectando ao Milvus em localhost:19540...
✓ Coleção 'lfw_cosface_modular' vinculada.
-> Carregando embeddings de teste para a memória...
✓ 13233 embeddings carregados com sucesso de 'recursos/'.


In [21]:
def create_hnsw_index(coll):
    index_params = {
        "metric_type": "COSINE",
        "index_type": "HNSW",
        "params": {"M": 16, "efConstruction": 256}
    }
    print("-> Construindo índice HNSW...")
    start = time.time()
    coll.create_index(field_name="embedding", index_params=index_params)
    print(f"✓ Índice concluído em {time.time() - start:.2f}s")

create_hnsw_index(collection)

-> Construindo índice HNSW...
✓ Índice concluído em 1.20s


In [22]:
def perform_search(coll, query_vectors, top_k=3):
    coll.load()
    start_time = time.time()
    
    # Busca solicitando os metadados person_id e image_path
    results = coll.search(
        data=query_vectors.tolist(), 
        anns_field="embedding", 
        param={"metric_type": "COSINE", "params": {"ef": 64}}, 
        limit=top_k,
        output_fields=["person_id", "image_path"]
    )
    
    latency = (time.time() - start_time) / len(query_vectors)
    
    print(f"⚡ Latência média: {latency*1000:.2f} ms")
    
    # Exemplo formatado do primeiro resultado
    first_hit = results[0][0]
    print(f"\nMelhor Correspondência:")
    print(f"ID: {first_hit.entity.get('person_id')}")
    print(f"Caminho: {first_hit.entity.get('image_path')}")
    print(f"Score de Similaridade: {first_hit.distance:.4f}")

# Testar com as primeiras 100 queries
perform_search(collection, emb_data[:100])

⚡ Latência média: 63.23 ms

Melhor Correspondência:
ID: Rustu_Recber/Rustu_Recber_0001.jpg
Caminho: local_path/Rustu_Recber/Rustu_Recber_0001.jpg.jpg
Score de Similaridade: 1.0000
